In [ ]:
!pip install xgboost tensorflow scikit-learn


In [ ]:
import pandas as pd

# Load the file
prosodic_df = pd.read_csv("prosodic_features.csv")

# Extract participant ID (e.g., 'P1Q1' → 'P1')
prosodic_df["participant_id"] = prosodic_df["participant&question"].str.extract(r'(P\d+)')

# Select only numeric columns
numeric_cols = prosodic_df.select_dtypes(include=["number"]).columns

# Group by participant ID and average numeric features
prosodic_grouped = prosodic_df.groupby("participant_id")[numeric_cols].mean().reset_index()

# Save and download
prosodic_grouped.to_csv("prosodic_grouped.csv", index=False)

# Download in Colab
from google.colab import files
files.download("prosodic_grouped.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load the cleaned datasets
prosodic_grouped = pd.read_csv("prosodic_grouped.csv")  # Save this earlier
text_df = pd.read_csv("features.csv")
combined_df = pd.read_csv("combined_features_dataset.csv")

# Clean ID and align
prosodic_grouped["id"] = prosodic_grouped["participant_id"].str.lower()
text_df = text_df.sort_values(by="id").reset_index(drop=True)
combined_df = combined_df.sort_values(by="id").reset_index(drop=True)
merged_prosodic = prosodic_grouped.drop(columns=["participant_id"])

# Get common IDs
common_ids = set(merged_prosodic["id"]) & set(text_df["id"]) & set(combined_df["id"])
merged_prosodic = merged_prosodic[merged_prosodic["id"].isin(common_ids)].sort_values("id").reset_index(drop=True)
text_df = text_df[text_df["id"].isin(common_ids)].sort_values("id").reset_index(drop=True)
combined_df = combined_df[combined_df["id"].isin(common_ids)].sort_values("id").reset_index(drop=True)

# Simulated normalized PHQ-8 target
np.random.seed(42)
target = pd.Series(np.random.uniform(0, 1, len(common_ids)), name="target")

# Final feature matrices
X_prosodic = merged_prosodic.drop(columns=["id"])
X_combined = combined_df.drop(columns=["id"])


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from tensorflow.keras import models, layers, callbacks, optimizers, regularizers
from scipy.stats import pearsonr
import pandas as pd
import numpy as np

# Assume X_prosodic, X_combined, and target are already loaded and cleaned

# Normalize input
scaler_p = MinMaxScaler()
X_prosodic_scaled = scaler_p.fit_transform(X_prosodic)
scaler_c = MinMaxScaler()
X_combined_scaled = scaler_c.fit_transform(X_combined)

# Train/test split
Xp_train, Xp_test, yp_train, yp_test = train_test_split(X_prosodic_scaled, target, test_size=0.3, random_state=42)
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_combined_scaled, target, test_size=0.3, random_state=42)

def run_xgb(X_train, X_test, y_train, y_test):
    model = XGBRegressor(objective="reg:squarederror", random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return pearsonr(y_test, preds)[0], mean_absolute_error(y_test, preds)

def run_mlp(X_train, X_test, y_train, y_test):
    model = models.Sequential([
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(X_train.shape[1],)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizers.Adam(learning_rate=0.0005), loss='mean_squared_error')
    es = callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model.fit(X_train, y_train, validation_split=0.2, epochs=200, batch_size=8, callbacks=[es], verbose=0)
    preds = model.predict(X_test).flatten()
    return pearsonr(y_test, preds)[0], mean_absolute_error(y_test, preds)

r1, re1 = run_xgb(Xp_train, Xp_test, yp_train, yp_test)
r2, re2 = run_xgb(Xc_train, Xc_test, yc_train, yc_test)
r3, re3 = run_mlp(Xp_train, Xp_test, yp_train, yp_test)
r4, re4 = run_mlp(Xc_train, Xc_test, yc_train, yc_test)

pd.DataFrame({
    "Model": ["XGBoost (Prosodic)", "XGBoost (Multimodal)", "MLP (Prosodic)", "MLP (Multimodal)"],
    "Pearson r": [r1, r2, r3, r4],
    "Relative Error (MAE)": [re1, re2, re3, re4]
})


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


,Model,Pearson r,Relative Error (MAE)
0,XGBoost (Prosodic),-0.054991,0.281111
1,XGBoost (Multimodal),0.057728,0.259119
2,MLP (Prosodic),-0.299838,0.293590
3,MLP (Multimodal),-0.125783,0.265411


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from scipy.stats import pearsonr

# Simulated placeholder data (replace with real cleaned data)
np.random.seed(42)
n_samples = 68
X_prosodic = pd.DataFrame(np.random.rand(n_samples, 34), columns=[f"p_feat_{i}" for i in range(34)])
X_combined = pd.DataFrame(np.random.rand(n_samples, 52), columns=[f"c_feat_{i}" for i in range(52)])
target = pd.Series(np.random.rand(n_samples), name="target")  # Replace with real target

# Normalize
scaler_p = MinMaxScaler()
X_prosodic_scaled = scaler_p.fit_transform(X_prosodic)
scaler_c = MinMaxScaler()
X_combined_scaled = scaler_c.fit_transform(X_combined)

# Feature selection
selector_p = SelectKBest(score_func=f_regression, k=20)
X_prosodic_selected = selector_p.fit_transform(X_prosodic_scaled, target)
selector_c = SelectKBest(score_func=f_regression, k=20)
X_combined_selected = selector_c.fit_transform(X_combined_scaled, target)

# Train/Test Split
Xp_train, Xp_test, yp_train, yp_test = train_test_split(X_prosodic_selected, target, test_size=0.3, random_state=42)
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_combined_selected, target, test_size=0.3, random_state=42)

# XGBoost with RandomizedSearch
param_dist = {
    "n_estimators": [30, 50, 70],
    "max_depth": [2, 3, 4, 5, 6]
}
xgb = XGBRegressor(objective="reg:squarederror", random_state=42)
rand = RandomizedSearchCV(xgb, param_distributions=param_dist, n_iter=5, scoring='neg_mean_absolute_error', cv=3, random_state=42)
rand.fit(Xp_train, yp_train)
best_xgb = rand.best_estimator_

# Ridge Regression
ridge = Ridge(alpha=1.0)
ridge.fit(Xp_train, yp_train)

# Evaluation
r_xgb, _ = pearsonr(yp_test, best_xgb.predict(Xp_test))
mae_xgb = mean_absolute_error(yp_test, best_xgb.predict(Xp_test))
r_ridge, _ = pearsonr(yp_test, ridge.predict(Xp_test))
mae_ridge = mean_absolute_error(yp_test, ridge.predict(Xp_test))

# Cross-validation score
cv_scores = cross_val_score(best_xgb, Xp_train, yp_train, cv=5, scoring='neg_mean_absolute_error')
cv_mae = -np.mean(cv_scores)

# Results
results_df = pd.DataFrame({
    "Model": ["XGBoost (Prosodic)", "Ridge Regression (Prosodic)"],
    "Pearson r": [r_xgb, r_ridge],
    "Relative Error (MAE)": [mae_xgb, mae_ridge],
    "CV MAE (XGB only)": [cv_mae, None]
})
print(results_df)


                         Model  Pearson r  Relative Error (MAE)  \
0           XGBoost (Prosodic)   0.098509              0.218616   
1  Ridge Regression (Prosodic)   0.331067              0.209197   

   CV MAE (XGB only)  
0           0.260259  
1                NaN  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from xgboost import XGBRegressor
from tensorflow.keras import models, layers, callbacks, optimizers, regularizers

# ------------------- Replace with Real Data -------------------
np.random.seed(42)
n_samples = 68
X_prosodic = pd.DataFrame(np.random.rand(n_samples, 34))
X_combined = pd.DataFrame(np.random.rand(n_samples, 52))
target = pd.Series(np.random.rand(n_samples), name="target")  # Replace with real PHQ-8

# ------------------- Normalization -------------------
scaler_p = MinMaxScaler()
X_prosodic_scaled = scaler_p.fit_transform(X_prosodic)

scaler_c = MinMaxScaler()
X_combined_scaled = scaler_c.fit_transform(X_combined)

# ------------------- Feature Selection -------------------
selector_p = SelectKBest(score_func=f_regression, k=20)
X_prosodic_selected = selector_p.fit_transform(X_prosodic_scaled, target)

selector_c = SelectKBest(score_func=f_regression, k=20)
X_combined_selected = selector_c.fit_transform(X_combined_scaled, target)

# ------------------- Train-Test Split -------------------
Xp_train, Xp_test, yp_train, yp_test = train_test_split(X_prosodic_selected, target, test_size=0.3, random_state=42)
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_combined_selected, target, test_size=0.3, random_state=42)

# ------------------- XGBoost Function -------------------
def train_xgb(X_train, y_train):
    param_dist = {
        "n_estimators": [30, 50, 70],
        "max_depth": [2, 3, 4, 5]
    }
    xgb = XGBRegressor(objective="reg:squarederror", random_state=42)
    search = RandomizedSearchCV(xgb, param_distributions=param_dist, n_iter=5,
                                 scoring='neg_mean_absolute_error', cv=3, random_state=42)
    search.fit(X_train, y_train)
    return search.best_estimator_

# ------------------- MLP Function -------------------
def train_mlp(X_train, y_train, X_test, y_test):
    model = models.Sequential([
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(X_train.shape[1],)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizers.Adam(learning_rate=0.0005), loss='mean_squared_error')
    es = callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model.fit(X_train, y_train, validation_split=0.2, epochs=200, batch_size=8, callbacks=[es], verbose=0)
    preds = model.predict(X_test).flatten()
    return preds

# ------------------- Train All Models -------------------
# XGBoost - Prosodic
xgb_prosodic = train_xgb(Xp_train, yp_train)
preds_xgb_p = xgb_prosodic.predict(Xp_test)
r_xgb_p, mae_xgb_p = pearsonr(yp_test, preds_xgb_p)[0], mean_absolute_error(yp_test, preds_xgb_p)

# XGBoost - Multimodal
xgb_combined = train_xgb(Xc_train, yc_train)
preds_xgb_c = xgb_combined.predict(Xc_test)
r_xgb_c, mae_xgb_c = pearsonr(yc_test, preds_xgb_c)[0], mean_absolute_error(yc_test, preds_xgb_c)

# MLP - Prosodic
preds_mlp_p = train_mlp(Xp_train, yp_train, Xp_test, yp_test)
r_mlp_p, mae_mlp_p = pearsonr(yp_test, preds_mlp_p)[0], mean_absolute_error(yp_test, preds_mlp_p)

# MLP - Multimodal
preds_mlp_c = train_mlp(Xc_train, yc_train, Xc_test, yc_test)
r_mlp_c, mae_mlp_c = pearsonr(yc_test, preds_mlp_c)[0], mean_absolute_error(yc_test, preds_mlp_c)

# ------------------- Results Summary -------------------
results = pd.DataFrame({
    "Model": ["XGBoost (Prosodic)", "XGBoost (Multimodal)", "MLP (Prosodic)", "MLP (Multimodal)"],
    "Pearson r": [r_xgb_p, r_xgb_c, r_mlp_p, r_mlp_c],
    "Relative Error (MAE)": [mae_xgb_p, mae_xgb_c, mae_mlp_p, mae_mlp_c]
})
print(results)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
                  Model  Pearson r  Relative Error (MAE)
0    XGBoost (Prosodic)   0.098509              0.218616
1  XGBoost (Multimodal)   0.113992              0.229888
2        MLP (Prosodic)   0.345271              0.208356
3      MLP (Multimodal)   0.044977              0.248883
